Sources  
- http://host.robots.ox.ac.uk/pascal/VOC/voc2007/ (pascal dataset)
- https://github.com/keshik6/pascal-voc-classification/tree/master (pascal github inspo)
- http://host.robots.ox.ac.uk/pascal/VOC/voc2007/devkit_doc_07-Jun-2007.pdf (pascal devkit)
- https://github.com/FabrizioDeSantis/Object-Detection-Manga109/tree/main (manga109 github inspo)
- manga109 dataset \
- manga109 libraries, api 
- manga109 paper 
- https://towardsdatascience.com/bounding-box-prediction-from-scratch-using-pytorch-a8525da51ddc 

Dependencies 

In [1]:
import manga109api_custom
from sklearn.model_selection import train_test_split
import torchvision.models as models
import torchvision.transforms as T 
import torch 
from dataset import CustomDataset
from torch.utils.data import DataLoader
import albumentations as A 
import pandas as pd

Load Data (fix paths)

In [5]:
# directory with Manga109 data 
manga109_root_dir = "./Manga109/Manga109_released_2023_12_07" # directory with Manga109 data

# custom parser to access Manga109 data 
p = manga109api_custom.Parser(root_dir=manga109_root_dir)
img_dict = p.load_all_images()

# create pandas dataframe from the dictionary
df = pd.DataFrame(img_dict)
print(df)

                                                img_path  width  height  \
0      C:\Users\Grace\Documents\Manga109-BoundingDete...   1654    1170   
1      C:\Users\Grace\Documents\Manga109-BoundingDete...   1654    1170   
2      C:\Users\Grace\Documents\Manga109-BoundingDete...   1654    1170   
3      C:\Users\Grace\Documents\Manga109-BoundingDete...   1654    1170   
4      C:\Users\Grace\Documents\Manga109-BoundingDete...   1654    1170   
...                                                  ...    ...     ...   
10125  C:\Users\Grace\Documents\Manga109-BoundingDete...   1654    1170   
10126  C:\Users\Grace\Documents\Manga109-BoundingDete...   1654    1170   
10127  C:\Users\Grace\Documents\Manga109-BoundingDete...   1654    1170   
10128  C:\Users\Grace\Documents\Manga109-BoundingDete...   1654    1170   
10129  C:\Users\Grace\Documents\Manga109-BoundingDete...   1654    1170   

              book_id                                             bboxes  \
0                ARMS  

Split and Stratify Data 

In [10]:
test_ratio = 0.2
val_ratio = 0.2

# split data into training/validation and testing sets
train_val_df, test_df = train_test_split(df, test_size=test_ratio, stratify=df['book_id'], random_state=42)

# further split into training and validation sets 
train_df, val_df = train_test_split(train_val_df, test_size=val_ratio, stratify=train_val_df['book_id'], random_state=42)

print(f"Total Images: {len(df)}")
print(f"Training Images: {len(train_df)}")
print(f"Validation Images: {len(val_df)}")
print(f"Testing Images: {len(test_df)}")

Total Images: 10130
Training Images: 6483
Validation Images: 1621
Testing Images: 2026


Transformations

In [3]:
# define training transformations
train_transformations = A.Compose([
    # resize? 
    #T.PILToTensor(),                            
    #T.ConvertImageDtype(torch.float),           
    A.HorizontalFlip(p=0.5),
    T.ColorJitter(brightness=(0.80, 1.20)),
    A.Rotate(limit=90,p=0.5),
    A.RandomCropFromBorders(crop_left=0.1, crop_right=0.1, crop_top=0.1, crop_bottom=0.1, always_apply=False, p=1.0)
], bbox_params=A.BboxParams(format='coco'))

# define validation transformations
val_transformations = T.Compose([
    # resize?
    #T.PILToTensor(),
    #T.ConvertImageDtype(torch.float)
])

Create Datasets & DataLoaders 

In [4]:
train_dataset = CustomDataset(train_images,train_transformations)
val_dataset = CustomDataset(val_images)

batch_size = 8
num_workers = 4

# image, target 
train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=True)
valid_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=num_workers)

Visualize Data - testing & validation images  

In [5]:
from dataset import visualize_image_with_boxes
import torchvision.transforms.functional as F

image, annotation = next(iter(train_loader))

ValueError: Caught ValueError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "C:\Users\Grace\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\_utils\worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
           ^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Grace\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\_utils\fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "C:\Users\Grace\Documents\Manga109-BoundingDetection\dataset.py", line 59, in __getitem__
    image, my_annotation = self.transform(image=image_np, bboxes=bboxes)
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Grace\AppData\Local\Programs\Python\Python312\Lib\site-packages\albumentations\core\composition.py", line 211, in __call__
    p.ensure_data_valid(data)
  File "C:\Users\Grace\AppData\Local\Programs

Define and Train Model 

In [ ]:
model = models.resnet34
# optimizer = something 
# loss function = something 
# epochs

- Validation Metrics 
- Testing Images Visualization 
- Testing 
- Testing Evaluation Metrics 